In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
# asset = 'jqdata.000036_XSHG|1d'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2020-10-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

hs300cs = trdb2py.trading2_pb2.Candle(
    ts=int(trdb2py.str2timestamp('2021-03-11', '%Y-%m-%d')),
    open=52170000,
    close=52170000,
    high=52170000,
    low=52170000,    
)

hs300cs1 = trdb2py.trading2_pb2.Candle(
    ts=int(trdb2py.str2timestamp('2021-03-12', '%Y-%m-%d')),
    open=52170000,
    close=52170000,
    high=52170000,
    low=52170000,    
)

hs300 = trdb2py.trading2_pb2.Candles(
    market='jqdata',
    symbol='000300_XSHG|1d',
    candles=[hs300cs, hs300cs1],
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
    offset=29,
    candles=[hs300],
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0, ignoreCache=True)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

我们还是拿沪深300指数，2013年5月1日到2020年9月30日的日线数据来做例子。

In [ ]:
ret = trdb2py.getAssetCandles2(trdb2cfg, asset, tsStart, tsEnd, indicators=['ta-ema.29'], offset=29, simCandle=hs300)
# print(ret) 
# ret
trdb2py.showAssetCandles2('沪深300指数', ret, indicators=['ta-ema.29'], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[4, calcweekday2val2(4, 4)],
)

buy1 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ta-ema.{}'.format(29)],
)

buy2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[1, calcweekday2val2(1, 4)],
    group=1,
)

buy3 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ta-ema.29'],
    group=1,    
)

sell0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[3],
)

sell1 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[1],
    strVals=['buy'],
)

sell2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[5],
    group=1,     
)

sell3 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[2],
    strVals=['buy'],
    group=1,     
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0, buy1, buy2, buy3])
s0.sell.extend([sell0, sell1, sell2, sell3])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell) 
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='周内效应混合策略',
    offset=29,
    candles=[hs300],
)  

pnlm = trdb2py.simTrading(trdb2cfg, p0)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)
sts = trdb2py.getFirstCtrlTs(pnlm)

trdb2py.showPNLs([pnlm, pnlBaseline], toImg=isStaticImg, isShowBuy=True, isShowSell=True, width=width, height=height, startTs=sts)

In [ ]:
npnl0 = trdb2py.clonePNLWithTs(pnlBaseline['pnl'], sts)
npnl1 = trdb2py.clonePNLWithTs(pnlm['pnl'], sts)

lstallpnl = [{'title': '沪深300', 'pnl': npnl0},
             {'title': '周内效应混合策略', 'pnl': npnl1}]

for v in lstallpnl:
    trdb2py.rebuildPNL(v['pnl'])
    
dflstallpnl = trdb2py.buildPNLReport(lstallpnl)

dflstallpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility']].sort_values(by='totalReturns', ascending=False)
# dfpnl = trdb2py.buildPNLReport([pnlm, pnlBaseline])

# # dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

# dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)